# PokeWhom Project

## Imports

In [78]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import ast
import itertools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

## Read Data and Clean

In [81]:
# jupyter notebook
#source venv/bin/activate
#docker build -t my-jupyter .
#features = pokemon descriptions, height, weight, stats?
#labels = type

def clean_type(type):
    types = type.strip("{}").split(",")
    types = [t.strip().lower() for t in types]
    if len(types) == 1:
        return types[0]
    return f"{types[0]}-{types[1]}"

def type_primary(type):
    types = type.strip("{}").split(",")
    types = [t.strip().lower() for t in types] 
    return types[0]

def type_secondary(type):
    types = type.strip("{}").split(",")
    types = [t.strip().lower() for t in types] 
    if len(types) == 2:
        return types[1]
    return "None"

def remove_stopwords(description):
    stop_words = set(stopwords.words("english"))
    words = description.split()
    filtered = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered)
    
nltk.download('stopwords')
pokemon = pd.read_csv("../Data/pokedex.csv")
pokemon["type_clean"] = pokemon["type"].apply(clean_type)
pokemon["type_primary"] = pokemon["type"].apply(type_primary)
pokemon["type_secondary"] = pokemon["type"].apply(type_secondary)
pokemon["desc_clean"] = pokemon["info"].str.lower().str.replace(r"[^a-z\s]", "", regex=True)
pokemon["desc_clean"] = pokemon["desc_clean"].apply(remove_stopwords)
pokemon.head(3)
#pokemon["type_primary"].value_counts().plot(kind="bar")
#there's a disproportianate amount of water and normal types?

        



[nltk_data] Downloading package stopwords to /home/db/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,name,height,weight,hp,attack,defense,s_attack,s_defense,speed,type,evo_set,info,type_clean,type_primary,type_secondary,desc_clean
0,1,bulbasaur,7,69,45,49,49,65,65,45,"{grass,poison}",1,A strange seed was planted on its back at birt...,grass-poison,grass,poison,strange seed planted back birth plant sprouts ...
1,2,ivysaur,10,130,60,62,63,80,80,60,"{grass,poison}",1,"When the bulb on its back grows large, it appe...",grass-poison,grass,poison,bulb back grows large appears lose ability sta...
2,3,venusaur,20,1000,80,82,83,100,100,80,"{grass,poison}",1,The plant blooms when it is absorbing solar en...,grass-poison,grass,poison,plant blooms absorbing solar energy stays move...


## Features

In [95]:
x = pokemon["desc_clean"] #features
y = pokemon["type_primary"] #labels
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=4, stratify=y
)

vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


## Label Encoding

In [96]:
label_train = y_train
label_test = y_test

encoder = LabelEncoder()
encoder.fit(pokemon["type_primary"])

label_train_enc = encoder.transform(label_train)
label_test_enc = encoder.transform(label_test)


## Model

In [98]:
model = LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear")
model.fit(X_train_vec, label_train_enc)
type_pred_enc = model.predict(X_test_vec)
type_pred = encoder.inverse_transform(type_pred_enc)

## Evaulate

In [99]:
print(classification_report(label_test, type_pred))

              precision    recall  f1-score   support

         bug       0.26      0.31      0.29        16
        dark       0.00      0.00      0.00         9
      dragon       0.00      0.00      0.00         7
    electric       0.86      0.50      0.63        12
       fairy       0.00      0.00      0.00         6
    fighting       1.00      0.12      0.22         8
        fire       0.83      0.38      0.53        13
      flying       0.00      0.00      0.00         2
       ghost       0.25      0.14      0.18         7
       grass       0.41      0.43      0.42        21
      ground       0.00      0.00      0.00         8
         ice       0.67      0.33      0.44         6
      normal       0.29      0.58      0.39        24
      poison       0.25      0.12      0.17         8
     psychic       0.40      0.50      0.44        12
        rock       0.23      0.25      0.24        12
       steel       0.00      0.00      0.00         7
       water       0.36    

/home/db/NLP/PokeWhom/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/db/NLP/PokeWhom/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/db/NLP/PokeWhom/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
